In [ ]:
import pandas as pd
import numpy as np
import re
from transformers import AutoModel, AutoTokenizer
import torch

In [ ]:
def CleanText(text):
    text = re.sub(r'''[\[|\]]''', "", text).split()
    text = np.array(text, dtype="float64")
    return text

# Normal dataset
df1 = pd.read_csv("all_data_with_identitiesEmbedded.csv")
df1['comment_text'] = df1['comment_text'].apply(lambda text: CleanText(text))
df1['toxicity'] = df1['toxicity'].apply(lambda text: round(text))

In [ ]:
df = pd.read_csv("../CSVFiles/all_data_with_identities.csv")

In [ ]:
sum(df['other_religions'])

In [ ]:
# Normal dataset
df = pd.read_csv("../CSVFiles/all_data_with_identities.csv")
df['toxicity'] = df['toxicity'].apply(lambda text: np.round(text >= 0.5))

df['male'] = df['male'].apply(lambda x: np.round(x >= 0.5))
df['female'] = df['female'].apply(lambda x: np.round(x >= 0.5))
df['LGBTQ'] = df['LGBTQ'].apply(lambda x: np.round(x >= 0.5))
df['christian'] = df['christian'].apply(lambda x: np.round(x >= 0.5))
df['muslim'] = df['muslim'].apply(lambda x: np.round(x >= 0.5))
df['black'] = df['black'].apply(lambda x: np.round(x >= 0.5))
df['white'] = df['white'].apply(lambda x: np.round(x >= 0.5))
df['other_religion'] = df['other_religion'].apply(lambda x: np.round(x >= 0.5))

df = df.loc[:, ["comment_text", "split", "toxicity", "male", "female", "LGBTQ", "christian", "muslim", "other_religion", "black", "white"]]



In [ ]:
idxLst = []
splitLst = []

idxLst.extend(df.query('other_religion>=0.5 and toxicity == 1').index[:30])
for i in range(10):
    splitLst.append('train')
    splitLst.append('test')
    splitLst.append('val')
    

idxLst.extend(df.query('other_religion>=0.5 and toxicity == 0').index[:30])
for i in range(10):
    splitLst.append('train')
    splitLst.append('test')
    splitLst.append('val')
    
    
toxicity = [1, 0]

for t in toxicity:
    count = 0
    iterater = 0
    currentDF = df.query('toxicity=='+str(t))
    currentDFIndexes = currentDF.index
    while count < 25000:
        if currentDFIndexes[iterater] not in idxLst:
            idxLst.append(currentDFIndexes[iterater])
            count+=1
            if count <= 22000:
                splitLst.append('train')
            elif count > 22000 and count <= 23500:
                splitLst.append('test')
            else:
                splitLst.append('val')
        iterater +=1
    


In [ ]:
idxLst = []
splitLst = []

idxLst.extend(df.query('other_religion>=0.5 and toxicity == 1').index[:30])
for i in range(10):
    splitLst.append('train')
    splitLst.append('test')
    splitLst.append('val')
    

idxLst.extend(df.query('other_religion>=0.5 and toxicity == 0').index[:30])
for i in range(10):
    splitLst.append('train')
    splitLst.append('test')
    splitLst.append('val')
    
    
print(len(idxLst))
print(len(splitLst))


#categories = ['male', 'christian', 'LGBTQ', 'muslim', 'female', 'black', 'white']
toxicity = [1, 0]

for t in toxicity:
    for cat in categories:
        
        count = 0
        iterater = 0
        
        currentDF = df.query(cat+'>=0.5 and toxicity=='+str(t))
        currentDFIndexes = currentDF.index
        while count < 4000:
            if currentDFIndexes[iterater] not in idxLst:
                idxLst.append(currentDFIndexes[iterater])
                count+=1
                if count <= 3000:
                    splitLst.append('train')
                elif count > 3000 and count <= 3500:
                    splitLst.append('test')
                else:
                    splitLst.append('val')
            iterater +=1
        print(iterater)

In [ ]:
newDF = df.loc[idxLst]

for i, idx in enumerate(idxLst):
    newDF.at[idx, 'split'] = splitLst[i]

In [ ]:
newDF.to_csv(r'C:\Users\frede\Desktop\small50000AllDataWithIdentities.csv', index=False, header=True)

In [ ]:
tempDF = newDF.loc[:, ["comment_text", "split", "toxicity", "male", "female", "LGBTQ", "christian", "muslim", "other_religion", "black", "white"]]

print('Toxic samples: ', sum(newDF['toxicity']))
print('None-toxic samples: ', len(newDF['toxicity'])-sum(newDF['toxicity']))

print("\n")

print("Validation samples: ", np.sum(newDF['split'] == 'val'))
print("Test samples", np.sum(newDF['split'] == 'test'))

print("\n")

for col in tempDF.columns[3:]:
    print(col + ": " + str(np.sum(tempDF[col])))

In [ ]:
import pandas as pd
import numpy as np
import re
from transformers import AutoModel, AutoTokenizer
import torch



newDF = newDF.loc[:, ["comment_text", "split", "toxicity", "male", "female", "LGBTQ", "christian", "muslim", "other_religion", "black", "white"]]

newDF['male'] = newDF['male'].apply(lambda x: np.round(x >= 0.5))
newDF['female'] = newDF['female'].apply(lambda x: np.round(x >= 0.5))
newDF['LGBTQ'] = newDF['LGBTQ'].apply(lambda x: np.round(x >= 0.5))
newDF['christian'] = newDF['christian'].apply(lambda x: np.round(x >= 0.5))
newDF['muslim'] = newDF['muslim'].apply(lambda x: np.round(x >= 0.5))
newDF['black'] = newDF['black'].apply(lambda x: np.round(x >= 0.5))
newDF['white'] = newDF['white'].apply(lambda x: np.round(x >= 0.5))
newDF['other_religion'] = newDF['other_religion'].apply(lambda x: np.round(x >= 0.5))


dfToEmbedding= newDF

bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)                     
bertweet.eval()
bertweet.to('cuda')

with torch.no_grad():
    for i, idx in enumerate(dfToEmbedding.index):
        bertweet.resize_token_embeddings(len(tokenizer))
        dfToEmbedding.at[idx, 'comment_text'] = str(bertweet(torch.tensor(tokenizer.encode
                                                                  (dfToEmbedding['comment_text'][idx], add_special_tokens=True,
                                                                   truncation=True)).to('cuda').unsqueeze(0))[1].detach().cpu().numpy()[0])

dfToEmbedding.to_csv(r'C:\Users\frede\Desktop\50000DomainDataBertweetEmbedded.csv', index=False, header=True)

In [ ]:
import pandas as pd
import numpy as np
import re
from transformers import AutoModel, AutoTokenizer
import torch

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

dfToEmbedding= pd.read_csv("../CSVFiles/smallDomainDataNotEmbedded.csv")


bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)                     
bertweet.eval()
bertweet.to('cuda')

with torch.no_grad():
    for i, idx in enumerate(dfToEmbedding.index):
        embeds = bertweet(torch.tensor(tokenizer.encode(dfToEmbedding['comment_text'][idx], add_special_tokens=True,
                                                                   truncation=True)).to('cuda').unsqueeze(0))[0].squeeze(0)
        
        if embeds.shape[0] < 200:
            embeds = torch.cat((embeds, torch.zeros((200-embeds.shape[0], 768)).to('cuda')))
            
        elif embeds.shape[0] > 200:
            embeds = embeds[0:200]
        
        dfToEmbedding.at[idx, 'comment_text'] = embeds.detach().cpu().numpy()